In [1]:
# ✅ STEP 1: Import ไลบรารี
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold

In [4]:
  # ✅ STEP 1: แตกไฟล์ ZIP ที่อัปโหลด
import zipfile

zip_path = r"D:\miewmiewfrom_dc\Testhack\HackTemp\individual-test-temperature-prediction.zip"  # แก้ตรงนี้ถ้าอัปโหลดใช้ชื่ออื่น
extract_path = "temp_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ แตกไฟล์เสร็จแล้ว")


✅ แตกไฟล์เสร็จแล้ว


In [5]:
train = pd.read_csv(r"D:\miewmiewfrom_dc\Testhack\HackTemp\temp_data\IOT_Train.csv")
test = pd.read_csv(r"D:\miewmiewfrom_dc\Testhack\HackTemp\IOT_Test.csv")

X_train = train.drop(columns=["temp"])
y_train = train["temp"]
X_test = test.drop(columns=["id"])

In [6]:
from sklearn.preprocessing import LabelEncoder

# รวม train และ test เพื่อลง encoding พร้อมกัน
X_all = pd.concat([X_train, X_test], axis=0)
X_all_encoded = X_all.copy()

# ค้นหาคอลัมน์ที่เป็น object (string)
cat_cols = X_all.select_dtypes(include='object').columns

# แปลงค่าด้วย LabelEncoder
for col in cat_cols:
    le = LabelEncoder()
    X_all_encoded[col] = le.fit_transform(X_all[col].astype(str))

# แยกกลับออกเป็น train/test
X_train_encoded = X_all_encoded.iloc[:len(X_train)]
X_test_encoded = X_all_encoded.iloc[len(X_train):]


In [7]:
# ✅ STEP 3: Cross Validation + LightGBM Training
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
import numpy as np
import lightgbm as lgb

kf = KFold(n_splits=5, shuffle=True, random_state=42)
models = []
mae_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_encoded, y_train)):
    X_tr, X_val = X_train_encoded.iloc[train_idx], X_train_encoded.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    model = lgb.LGBMRegressor(n_estimators=500, learning_rate=0.03, random_state=fold)
    model.fit(X_tr, y_tr)

    val_pred = model.predict(X_val)

    # ✅ กรอง NaN ออกก่อนวัด MAE
    mask = ~np.isnan(val_pred) & ~np.isnan(y_val.values)
    val_pred_clean = val_pred[mask]
    y_val_clean = y_val.values[mask]

    mae = mean_absolute_error(y_val_clean, val_pred_clean)
    mae_scores.append(mae)
    models.append(model)

    print(f"Fold {fold+1} MAE: {mae:.4f}")

# ✅ เฉลี่ยค่า MAE ทั้ง 5 fold
print(f"\n✅ Avg MAE: {sum(mae_scores)/len(mae_scores):.4f}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000675 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1015
[LightGBM] [Info] Number of data points in the train set: 10908, number of used features: 18
[LightGBM] [Info] Start training from score 29.073662
Fold 1 MAE: 0.5363
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000383 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1013
[LightGBM] [Info] Number of data points in the train set: 10908, number of used features: 18
[LightGBM] [Info] Start training from score 29.090722
Fold 2 MAE: 0.5434
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000362 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info

In [9]:
# ✅ STEP 4: ทำนายด้วยค่าเฉลี่ยจากหลายโมเดล (Ensemble)
import numpy as np

predictions = np.mean([m.predict(X_test_encoded) for m in models], axis=0)

submission = pd.DataFrame({
    "id": test["id"],
    "temp": predictions
})
submission.to_csv("Submitiontemp001.csv", index=False)
print("✅ submission.csv สร้างเสร็จแล้ว")


✅ submission.csv สร้างเสร็จแล้ว
